In [37]:
import numpy as np
import pandas as pd
#чтобы полный текст посмотреть
pd.set_option('display.max_colwidth', None)
from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from pymorphy2 import MorphAnalyzer
# лемматизация и мешок слов
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
#удаляю русские стоп слова (расширить???)
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
import spacy
from spacy.lang.ru import Russian

In [38]:
main_data = pd.DataFrame(pd.read_excel('../datasets/ai_bot_app_post.xls'))

In [39]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88182 entries, 0 to 88181
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88182 non-null  int64 
 1   telegram_id  88182 non-null  int64 
 2   text         88178 non-null  object
 3   title        88177 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.7+ MB


In [40]:
main_data = main_data.apply(lambda x: x.astype(str).str.lower()).drop_duplicates(subset=['text', 'title'], keep='first')
main_data.info()
#ушло 11747 13%, пустые тоже вылетают 

<class 'pandas.core.frame.DataFrame'>
Index: 76435 entries, 0 to 88180
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      76435 non-null  object
 1   telegram_id  76435 non-null  object
 2   text         76435 non-null  object
 3   title        76435 non-null  object
dtypes: object(4)
memory usage: 2.9+ MB


In [41]:
#ch_nan = main_data['text'].isnull().values.any()
#print(ch_nan)

In [42]:
#случайным образом выбираем 10 % новостей
main_data = main_data.sample(frac=0.1)
main_data.head(1)

,user_id,telegram_id,text,title
76645,76840,31461,"**залужный: ""контрнаступление всу – не шоу"". беларусь может принять до 8 тысяч ""вагнеровцев""**\nhttps://ru.euronews.com/2023/07/01/ru-ukraine-update?utm_source=telegram.messenger&utm_campaign=feeds_bcs_topstories&utm_medium=referral","**залужный: ""контрнаступление всу – не шоу"""


In [43]:
#main_data.info()

In [44]:
#main_data = main_data.iloc[0:30]

In [45]:
#df_new.head(1)

In [46]:
main_data['text_clean'] = main_data['text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
#df_new['text_clean'] = df_new['text_clean'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
#df_new['text_clean'].head(3)

In [47]:
main_data['text_clean'] = main_data['text_clean'].str.replace('предложить новость', '')
main_data['text_clean'] = main_data['text_clean'].str.replace('подписаться', '')
main_data['text_clean'] = main_data['text_clean'].str.replace('SUBSCRIBE', '')
main_data['text_clean'] = main_data['text_clean'].str.replace('НАСТОЯЩИЙ МАТЕРИАЛ (ИНФОРМАЦИЯ) ПРОИЗВЕДЕН, РАСПРОСТРАНЕН И (ИЛИ) НАПРАВЛЕН ИНОСТРАННЫМ АГЕНТОМ', '')
main_data['text_clean'] = main_data['text_clean'].str.replace('ЛИБО КАСАЕТСЯ ДЕЯТЕЛЬНОСТИ ИНОСТРАННОГО АГЕНТА', '')
main_data['text_clean'] = main_data['text_clean'].str.replace('\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '')
main_data['text_clean'] = main_data['text_clean'].str.replace('\n', '')
main_data['text_clean'] = main_data['text_clean'].str.replace('\t', ' ')
main_data['text_clean'] = main_data['text_clean'].str.replace(' {2,}', ' ', regex=True)
main_data['text_clean'] = main_data['text_clean'].str.strip()
#df_new.head(3)   нужно ли это внизу?????
main_data['text_clean'] = main_data['text_clean'].str.replace('\d+', '', regex=True)

In [48]:
main_data['text_clean'] = main_data['text_clean'].replace(r'[^\w\s]',' ',regex=True).replace(r'\s+',' ',regex=True).str.lower()
#df_new['text_clean'] = df_new['text'].replace(r'[^\w\s]',' ',regex=True).replace(r'\s+',' ',regex=True).str.lower()
#df_new.head(3)

In [49]:
main_data.head(1)

,user_id,telegram_id,text,title,text_clean
76645,76840,31461,"**залужный: ""контрнаступление всу – не шоу"". беларусь может принять до 8 тысяч ""вагнеровцев""**\nhttps://ru.euronews.com/2023/07/01/ru-ukraine-update?utm_source=telegram.messenger&utm_campaign=feeds_bcs_topstories&utm_medium=referral","**залужный: ""контрнаступление всу – не шоу""",залужный контрнаступление всу не шоу беларусь может принять до тысяч вагнеровцев


In [50]:
import re
def remove_stop_words(text):
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in russian_stopwords]
    return text

main_data['text_clean'] = main_data['text_clean'].apply(lambda x: remove_stop_words(str(x)))

In [51]:
#nlp = spacy.load('ru_core_news_md')
def tokenize_n_normalize(sent, pat=r"(?u)\b\w\w+\b", morph=MorphAnalyzer()):
    return [morph.parse(tok)[0].normal_form 
            for tok in regexp_tokenize(sent, pat)]

main_data['text_lemma'] = main_data['text_clean'].map(lambda x: " ".join(tokenize_n_normalize(str(x))))

In [52]:
main_data.head(1)

,user_id,telegram_id,text,title,text_clean,text_lemma
76645,76840,31461,"**залужный: ""контрнаступление всу – не шоу"". беларусь может принять до 8 тысяч ""вагнеровцев""**\nhttps://ru.euronews.com/2023/07/01/ru-ukraine-update?utm_source=telegram.messenger&utm_campaign=feeds_bcs_topstories&utm_medium=referral","**залужный: ""контрнаступление всу – не шоу""","[, залужный, контрнаступление, всу, шоу, беларусь, принять, тысяч, вагнеровцев, ]",залужный контрнаступление всу шоу беларусь принять тысяча вагнеровец


In [17]:
main_data.to_csv('../datasets/ready_to_work.csv')

In [18]:
main_data.to_excel("../datasets/ready_to_work.xlsx")